# Práctica 1 - Clasificación de palabras

## Introducción
El problema que se nos plantea en esta práctica es la clasificación, mediante un modelo basado en SVM, de las palabras de un dataset según la lengua a la que pertenecen. En primera instáncia solamente tenemos palabras del catalán y el inglés, pero se pueden añadir más palabras de otros idiomas usando librerías como [Googletrans](https://www.thepythoncode.com/article/translate-text-in-python#:~:text=Googletrans%20is%20a%20free%20and,detect%20languages%20and%20translate%20text.). 

In [136]:
# Misc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import mean
from matplotlib.colors import ListedColormap
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

# CrossValidation
## K-folding
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

## GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

#https://scikit-learn.org/stable/modules/model_evaluation.html
# Model
from sklearn.svm import SVC

## Preparación de los datos
En esta primera parte de la práctica vamos a preparar los datos para poder hacer la clasificación. Para ello haremos que cada muestra siga el siguiente formato: $C_1$ | $C_2$ | $C_3$ | .. | $C_n$ | $y$  (siendo $C$ una característica y $y$ el target).

Se va a codificar $y$ con los valores "0" para representar la clase _catala_ y "1" para la clase _angles_. Si añadieramos más idiomas, simplemente seguiríamos con este formato, siendo por ejemplo _francés_ "2", _alemán_ "3", etc. Esto se conoce como [Label Encoding](https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd). Este tipo de codificación de variables categóricas nos viene como anillo al dedo, ya que no puede dar lugar a la confusión de que los números representan una jerarquía, debido a que estamos tratando con lenguas modernas (la jerarquía podría indicar lo cerca que está esa lengua de su lengua madre, el latín por ejemplo). 

La otra opción sería llevar a cabo Hot Encoding, lo que supondría crear más variables $y$ con los valores 1 y 0 para indicar si pertenece una palabra a un idioma. Esta aproximación nos podría acabar incrementando muchísimo la dimensionalidad del problema.

In [137]:
#Auxiliar function that reformats df to one that is more fitting to Classification Problems
def reformat(dataFrame):
    dataFrame['y']=dataFrame.columns[0]
    
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame

#RawData, it has been a little bit formatted before reading the csv, to facilitate the process.
raw=pd.read_csv("data/data.csv")
#Split df
catala, angles= raw.filter(['catala'], axis=1), raw.filter(['angles'], axis=1)
#Reformating
catala=reformat(catala)
angles=reformat(angles)

#Merging
wordsDF=pd.concat([catala,angles], axis=0)

#Encoding variables
wordsDF['y']=wordsDF['y'].astype('category')
wordsDF['y']=wordsDF['y'].cat.codes
#Shuffle the rows
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)



### Caracterísiticas
Las características que hemos pensado que pueden ser útiles para la clasificación de las palabras son:
- Cantidad de caracteres (Númerica)
- Proporción de consonantes por vocal (consonantes / vocales) (Numérica)
- Contiene patrones o normas ortográficas de una lengua de las que vamos a clasificar?
    + Doble uso de vocal consecutivamente como es el caso del inglés (Categórica)
    + Acentos en caso de catalán (Categórica)
    + Contiene combinaciones de consonantes (consonant clusters) propias del inglés? (Categórica)

Referencias
- <[Consonant_Clusters](https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters)>
- <[Frecuencia_De_Letras_Usadas_En_Catalan](https://es.sttmedia.com/frecuencias-de-letras-catalan)>
- <[Frecuencia_De_Letras_Usadas_En_Catalan](https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html)>

Para añadir las columnas que representen estas características, hemos aplicado las siguientes funciones a las palabras.

In [138]:
#ratio de consonantes y vocales
def ratio (word):
    vocals=0
    for c in word:
        if isVocal(c):
            vocals+=1
    return round(vocals/len(word), 4)

#isVocal?
def isVocal(c):
    if(c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
        return True
    return 
#gotAccent?
def gotAccent(word):
    #List containing all possible accentuated chars from Catalan
    accentuatedChars=[ord('à'),ord('è'),ord('é'),ord('í'),ord('ò'),ord('ó'),ord('ú')]
    for c in word:
        if ord(c) in accentuatedChars:
            return 1
    return 0
def doubleVocal(word):
    ocurrences=["aa","ee","ii","oo","uu"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

def enCC(word):
    ocurrences=["sch","spl","shr","squ","thr","spr","scr","sph","th","tw","sw","sk","sm"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

#Features Adding
wordsDF['ratio']=wordsDF['word'].apply(ratio)
wordsDF['cantidadLetras']=wordsDF['word'].apply(len)
wordsDF['gotAccent']=wordsDF['word'].apply(gotAccent)
wordsDF['doubleVocal']=wordsDF['word'].apply(doubleVocal)
wordsDF['enCC']=wordsDF['word'].apply(enCC)
#Reorganize DF
wordsDF=wordsDF[['word','ratio','cantidadLetras','gotAccent','doubleVocal','enCC','y']]
#Write DF to csv
wordsDF.to_csv('data/definitiveData.csv', index=False)
#Checking
print(wordsDF)

          word   ratio  cantidadLetras  gotAccent  doubleVocal  enCC  y
0        spend  0.2000               5          0            0     0  0
1       mercat  0.3333               6          0            0     0  1
2          cas  0.3333               3          0            0     0  1
3      observe  0.4286               7          0            0     0  0
4     insectes  0.3750               8          0            0     0  1
...        ...     ...             ...        ...          ...   ... ..
1971      they  0.2500               4          0            0     1  0
1972    planet  0.3333               6          0            0     0  0
1973    people  0.5000               6          0            0     0  0
1974    speech  0.3333               6          0            1     0  0
1975       run  0.3333               3          0            0     0  0

[1976 rows x 7 columns]


### Separación del conjunto de datos en entrenamiento y test
Vamos a escoger dos tercios para el conjunto de entrenamiento y el resto para el test.
 Puntualizar que se han normalizado los datos para un mejor rendimiento del modelo. Solo se han escalado los datos numéricos, ya que sería erróneo escalar todo el dataset, con los categóricos, ya que podría dar lugar a dar menos importancia a estos últimos. 

In [139]:
#First we must separate dataframe into X and y format
X=wordsDF.iloc[:,1:6]
X_aux=wordsDF.iloc[:,1:3]
y=wordsDF.iloc[:,6]

#For better perfomance, we scale the data using an standard scaler
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(X_aux)
standardData_aux = scaler.transform(X_aux)

#Standarized DF
standardData=X
#Substitute standarized values in correspondent columns
standardData['ratio']=standardData_aux[:,0]
standardData['cantidadLetras']=standardData_aux[:,1]

#Then we separate the data frame in training and test (will be used in chosen model)
X_train, X_test, y_train, y_test = train_test_split(standardData, y, test_size=0.33, random_state=33)

## Modelos
### Primer Modelo

En este problema creemos que el mejor modelo para clasificar las palabras va a ser una SVC con un kernel gaussiano (__rbf__) ya que tenemos un número de características bastante bajo en comparación al número de muestras. Se han hecho pruebas con un kernel líneal, y si bien los resultados se acercan, el kernel gaussiano ha dado mejores resultados de media.

Los híperparámetros que deberemos de configurar en el modelo del SVM son:
- _C_: El parámetro __C__ en una SVM es un hiperparámetro que controla __la flexibilidad__ del modelo. Se utiliza para controlar el trade-off entre la complejidad del modelo y la cantidad de errores de clasificación que se permiten. Si C es un valor alto, el modelo se vuelve menos flexible y puede tender al overfitting. Si C es un valor bajo, el modelo es más flexible y permite más errores de clasificación. 

- _max\_iter_: __Número máximo de iteraciones__ que se llevará acabo en el entrenamiento del modelo, para encontrar una solución óptima. Si el parámetro es muy pequeño puede que no encuentre una solución óptima y tenga un rendimiento bajo. Por lo contrario si ponemos un valor muy alto podría alargar el tiempo de entrenamiento innecesariamente, ya que una solución óptima se podría encontrar con menos iteraciones.

- _gamma_: Controla el __ancho del kernel__. Cuanto mayor es su valor, menor es el ancho del kernel y viceversa. Si el kernel es muy ancho el modelo se vuelve más suave y no es tan sensible a los detalles de las muestras. 

El resto de híperparámetros no son tan necesarios ya que no se ajustan al problema. (Explicar algún que otro parámetro y decir por qué no es útil)

In [140]:
#Model
SVM_rbf = SVC(kernel='rbf')

### Nested Cross-Validation
Para elegir los mejores valores para los híperparámetros y a su vez elegir el modelo que mejor generaliza debemos de hacer uso de la técnica __Nested Cross-validation__. Esta técnica consiste en dos bucles, uno exterior y otro interior:
- En el interior se aplica __GridSearch__, donde se extrae la configuración de híperparámetros que mejores resultados da.
- En el exterior se hace un reentrenamiento con estos híperparámetros y se prueba el modelo con el conjunto de test correspondiente.

Al final del proceso tendremos K modelos, con el score (_accuracy_ normalmente) correspondiente a cada uno.

Referencias:<br>
- <https://ploomber.io/blog/nested-cv/>
- En este artículo se demuestra que hacer solo cross-validation puede resultar en un error de generalización optimista (debido a overfitting) 
<https://jmlr.csail.mit.edu/papers/volume11/cawley10a/cawley10a.pdf>


In [141]:
#Hyper Grid
hyper_grid = {"C":[0.1,1,10],"max_iter":[100,1000,10000],"gamma":[.01,.1,1]}
#Inner and Outer crossvalidation, following steps from :
#https://www.analyticsvidhya.com/blog/2021/03/a-step-by-step-guide-to-nested-cross-validation/
# and
# https://inria.github.io/scikit-learn-mooc/python_scripts/cross_validation_nested.html

inner_cv=StratifiedKFold(n_splits=3, shuffle=True)
outer_cv=StratifiedKFold(n_splits=5, shuffle=True)
#GridSearch definition
search=GridSearchCV(estimator=SVM_rbf,cv=inner_cv,param_grid=hyper_grid,scoring="accuracy",n_jobs=-1)

#Test score after nesting
test_score=cross_val_score(search,X_train,y_train,cv=outer_cv,n_jobs=-1)
#search.fit(X_train,y_train)
#Printing results of Nested Cross-Validation
print(f"Mean score of Nested cross-validation:"f"{test_score.mean():.3f}+-{test_score.std():.3f}")
#print(f"The best estimator is:"f"{search.best_estimator_}")

Mean score of Nested cross-validation:0.652+-0.027


In [142]:
#CrossValidation
inner_cv_man=StratifiedKFold(n_splits=3, shuffle=True)
outer_cv_man=StratifiedKFold(n_splits=5, shuffle=True)

#GridSearch definition
search=GridSearchCV(estimator=SVM_rbf,cv=inner_cv,param_grid=hyper_grid,scoring="accuracy",n_jobs=-1)
aux_X=X_train.to_numpy()
aux_y=y_train.to_numpy()
for train, test in outer_cv_man.split(aux_X,aux_y):
    
    X_train_cv, y_train_cv = aux_X[train], aux_y[train]
    X_test_cv, y_test_cv = aux_X[test], aux_y[test]
    modelo = search.fit(X_train_cv, y_train_cv)
    
    print(modelo.best_estimator_)
    
    print(modelo.best_score_)

SVC(C=1, gamma=0.1, max_iter=1000)
0.6597454717143102
SVC(C=10, gamma=0.01, max_iter=1000)
0.6493502661172633


c:\Users\pablo\miniconda3\envs\aprenentatge_automatic_2223\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(C=10, gamma=0.1, max_iter=1000)
0.6616099021375225
SVC(C=1, gamma=0.1, max_iter=1000)
0.6553352219074599
SVC(C=10, gamma=0.01, max_iter=1000)
0.6525023607176582


### Prueba del modelo con los parámetros encontrados

Una vez hemos encontrado los mejores híperparámetros para el SVM con Kernel Gaussiano, procedemos a entrenarlo y a realizar la predicción.

In [143]:
#Definition of SVM with best hyperparamters
SVM_rbf=SVC(kernel="rbf",C=0.1,max_iter=1000)

#Train Model with Train set
SVM_rbf.fit(X_train,y_train)

#Predict with X_test
y_predicted=SVM_rbf.predict(X_test)
differences=y_predicted-y_test
errors = np.count_nonzero(differences)

#Print results
print(f'Rati d\'acerts en el bloc de predicció: {(len(y_predicted)-errors)/len(y_predicted)}')

Rati d'acerts en el bloc de predicció: 0.6309341500765697
